<a href="https://colab.research.google.com/github/kethlyncampos/webscraper/blob/main/web_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install requests
!pip install iso8601

In [54]:
import requests
import pandas as pd
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
import pytz
from datetime import datetime
import iso8601
from dateutil import tz
from google.colab import drive
from selenium.webdriver.common.by import By
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
drive.mount("/content/gdrive")

In [55]:
yahoo = pd.read_csv('/content/gdrive/MyDrive/news_yahoo.csv')
infomoney = pd.read_csv('/content/gdrive/MyDrive/news_infomoney.csv', lineterminator='\n')

In [56]:
comp_yahoo = []
comp_infomoney = []

for c in yahoo['URL']:
  comp_yahoo.append(c)

for c in infomoney['URL']:
  comp_infomoney.append(c)

In [97]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-notifications")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36")
browser = webdriver.Chrome(options=options)

In [75]:
def req_yahoo(url):
  browser.get(url)
  scrollar()
  links = browser.find_elements(By.CSS_SELECTOR, 'a.js-content-viewer.rapidnofollow')
  noticias = []

  for link in links:
    if (('https://br.yahoo.com/news/' not in link.get_attribute('href')) and (link.get_attribute('href') not in comp_yahoo)):
      browser.execute_script("arguments[0].click();", link)
      scrollar()
      WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.TAG_NAME, "h1")))
      temp = BeautifulSoup(browser.page_source, 'html.parser')
      title = temp.find('h1', attrs={'data-test-locator':'headline'}).text
      author = temp.find('span', attrs={'class':'caas-author-byline-collapse'}).text
      collect_date = convert_timezone(datetime.now())
      publication_date = convert_timezone(iso8601.parse_date(temp.find('time')['datetime']))
      description = temp.find('div', attrs={'class':'caas-body'})
      try:
        description.figure.decompose()
        description = description.text
      except:
        description = description.text
      noticias.append(['Yahoo Finanças',title, description, author, publication_date, collect_date, browser.current_url])
      browser.back()
      sleep(5)

  return noticias

In [102]:
def req_infomoney(url):
  browser.get(url)
  #browser.implicitly_wait(10)
  browser.maximize_window()
  alert = browser.switch_to.active_element.click
  infinite = browser.find_element(By.ID, 'infinite-handle')
  browser.execute_script("arguments[0].click();", infinite)
  count = 40
  while(count > 0):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.ID, "infinite-handle")))
    browser.execute_script("arguments[0].click();", infinite)
    newHeight = browser.execute_script("return document.body.scrollHeight")
    lastHeight = newHeight
    count-=1
  return BeautifulSoup(browser.page_source, 'html.parser')

In [60]:
def scrollar():
  lastHeight = browser.execute_script("return document.body.scrollHeight")
  while True:
      browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
      sleep(5)
      newHeight = browser.execute_script("return document.body.scrollHeight")
      if newHeight == lastHeight:
          break
      lastHeight = newHeight
  return

In [61]:
def convert_timezone(date_time):
  sp = tz.gettz('America/Sao_Paulo')
  new_date = date_time.replace(tzinfo=pytz.utc)
  new_date = new_date.astimezone(sp)
  new_date = new_date.strftime('%Y-%m-%d %H:%M:%S')
  return new_date

In [62]:
def trata_string(data):
  data = re.sub(r'[\t\n ]+', ' ', data)
  data = re.sub(r'(?s)<figure.*?</figure>', ' ', data)
  return data

In [ ]:
array_noticias = req_yahoo('https://br.yahoo.com/topics/bolsa-de-valores/')

In [82]:
dataset_temp = pd.DataFrame(array_noticias,columns=['Fonte', 'Título', 'Descrição', 'Autor', 'Data/Hora de Publicação', 'Data/Hora de Coleta', 'URL'])

In [ ]:
if dataset_temp > 0:
  dataset_temp.drop_duplicates(subset=['Descrição'], inplace = True)
  dataset_temp.reset_index(drop = True, inplace = True)
  yahoo = pd.concat([dataset_temp,yahoo],ignore_index=True)
  yahoo.to_csv('/content/gdrive/MyDrive/news_yahoo.csv')

In [103]:
page = req_infomoney('https://www.infomoney.com.br/mercados/')

In [104]:
links = page.find_all(class_="hl-title")

In [106]:
lista_url = []
for j in links:
  aux = j.find('a')
  if(aux):
    if "stock-pickers" not in aux['href']:
      lista_url.append(aux['href'])
lista_url = set(lista_url)

In [ ]:
noticias = []
subtitle = ""
for k in lista_url:
  if k not in comp_infomoney:
    response = requests.get(k)
    temp = BeautifulSoup(response.content, 'html.parser')
    title = temp.find('h1', attrs={'class': 'page-title-1'})
    if title: # se não houver título, a página é do tipo cotações e possui apenas as informações sobre a empresa
      title = title.text
      subtitle = trata_string(temp.find('p', attrs={'class': 'article-lead'}).text)
      author = temp.find('span', attrs={'class':'author-name'})
      if author:
        author = trata_string(author.find('a').text) # se não houver autor, a página é do tipo guias
        collect_date = convert_timezone(datetime.now())
        publication_date = iso8601.parse_date(temp.find('time', attrs={'class':'entry-date'})['datetime']).strftime('%Y-%m-%d %H:%M:%S')
        description = temp.find('div', attrs={'class':'article-content'})
        try:
          description.figure.decompose()
          description = description.text
        except:
          description = description.text
        description = trata_string(description)
        noticias.append(['Infomoney',title, subtitle, description, author, publication_date, collect_date, k])

In [112]:
dataset_temp = pd.DataFrame(noticias,columns=['Fonte', 'Título', 'Subtítulo','Descrição', 'Autor', 'Data/Hora de Publicação', 'Data/Hora de Coleta', 'URL'])

In [115]:
if len(dataset_temp > 0):
  dataset_temp.drop_duplicates(subset=['Descrição'], inplace = True)
  dataset_temp.reset_index(drop = True, inplace = True)
  infomoney = pd.concat([dataset_temp, infomoney])
  infomoney.to_csv('/content/gdrive/MyDrive/news_infomoney.csv')